# Experimentation

Purpose: show and communicate about the benefits of formal languages with description logics semantics

Mission: verify the integrity of a system model

Objectives: 
- use pattern matching to detect the inconsistencies,
- use graph analysis to display correction suggestions,
- allocate the correction of the system model to a user, selected as being familiar with the CAESAR WorkBench v1.23,
- assume that the test is finished when the graph analysis returns that the input model complies with the 3 different rules.

Note 1: the RDF dataset shall be 
- renamed: *.trig
- uploaded on Fuseki : http://localhost:3030/dataset.html?tab=upload&ds=/WBtest



Note 2: if you want to use a different endpoint, you would have to modify the following endpoint variables with "http://localhost:3030/yourDataset/query"
- workpackageQueryEndpoint
- powerAssemblyQueryEndpoint
- powerInterfaceQueryEndpoint

In [58]:
from SPARQLWrapper import SPARQLWrapper, JSON
import pandas as pd

In [59]:
def SPARQL_Run(Endpoint, Query):
    SPARQL = SPARQLWrapper(Endpoint)   
    SPARQL.setQuery(Query) 
    SPARQL.setReturnFormat(JSON)
    return SPARQL.query().convert() 

# Rule #1 : Each subsystem shall be supplied by exactly one work package

In [60]:
endpoint = SPARQLWrapper("http://localhost:3030/WBv13.4")
endpoint.setQuery("""

# Each subsystem shall be supplied by exactly one work package
# List all the subsystems, and as an option: the work packages that supply them 

PREFIX project: <http://imce.jpl.nasa.gov/foundation/project/project#>
PREFIX oml: <http://def.seegrid.csiro.au/ontology/om/om-lite#>
PREFIX owl:   <http://www.w3.org/2002/07/owl#>
PREFIX rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX fse:   <http://imce.jpl.nasa.gov/discipline/fse/fse#>
PREFIX base:  <http://imce.jpl.nasa.gov/foundation/base/base#>
PREFIX analysis: <http://imce.jpl.nasa.gov/foundation/analysis/analysis#>
PREFIX mission: <http://imce.jpl.nasa.gov/foundation/mission/mission#>
PREFIX pr: <http://purl.org/ontology/prv/core#>
PREFIX pro: <http://purl.org/hpi/patchr#>

SELECT DISTINCT ?subSystem ?workPackage

FROM <urn:x-arq:UnionGraph>
WHERE
{
  ?a a fse:Subsystem ;
     base:hasCanonicalName ?subSystem .
  
  optional {
    ?b a project:WorkPackage ;
       base:hasCanonicalName ?workPackage ;
       project:supplies ?a .
  } 
}
""")

endpoint.setReturnFormat(JSON)
results = endpoint.query().convert()['results']['bindings']
#print(results)
#type(results)

obj = pd.DataFrame(results)

my_data = {}
for key, value in obj.items():  
    current_dict = obj[key]
    print(current_dict)
    for current_key, current_value in current_dict.items():
        if (str(current_value)!="nan"):
            my_data[obj[key]]=current_value["value"]
            
for el in my_data: 
    print(el)


    

0       {'type': 'literal', 'value': 'Communications'}
1      {'type': 'literal', 'value': 'Thermal control'}
2    {'type': 'literal', 'value': 'Command and data...
3    {'type': 'literal', 'value': 'Structures and m...
4              {'type': 'literal', 'value': 'Payload'}
5           {'type': 'literal', 'value': 'Propulsion'}
6                  {'type': 'literal', 'value': 'EPS'}
7    {'type': 'literal', 'value': 'Firesat flight s...
8         {'type': 'literal', 'value': 'ADCS and GNC'}
Name: subSystem, dtype: object


TypeError: 'Series' objects are mutable, thus they cannot be hashed

# Rule #2 : Each power load assembly shall be connected to at least one power source assembly

Note:
1. Solar arrays provide power to the EPS regulators and converters
2. EPS regulators and converters provide power to the batteries
3. Batteries provide power to the EPS regulators and converters 
4. EPS regulators and converters provide power to all the other power load components

In [ ]:
powerAssemblyQuery <-
'
# Each power load assembly shall be connected to at least one power source assembly
# List all the power load assemblies

PREFIX oml: <http://def.seegrid.csiro.au/ontology/om/om-lite#>
PREFIX owl:   <http://www.w3.org/2002/07/owl#>
PREFIX rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX fse:   <http://imce.jpl.nasa.gov/discipline/fse/fse#>
PREFIX base:  <http://imce.jpl.nasa.gov/foundation/base/base#>
PREFIX analysis: <http://imce.jpl.nasa.gov/foundation/analysis/analysis#>
PREFIX mission: <http://imce.jpl.nasa.gov/foundation/mission/mission#>

SELECT DISTINCT ?powerLoadAssembly ?loadAssemblyID ?functionID ?powerSourceAssembly ?sourceAssemblyID
FROM <urn:x-arq:UnionGraph>
WHERE
{
  ?a a fse:Assembly ;
     base:hasAnnotations "Power load" ;
     fse:hasAssemblyReferenceDesignator ?loadAssemblyID ;                                
	 base:hasCanonicalName ?powerLoadAssembly .

  optional {
  ?d a fse:Assembly ;
     base:hasAnnotations "Power source" ;
     fse:hasAssemblyReferenceDesignator ?sourceAssemblyID ;
	 base:hasCanonicalName ?powerSourceAssembly .
  
  ?c a mission:Junction ;
     fse:hasFunctionNumber ?functionID ;
     fse:connectsAssembly ?a ;
     fse:connectsAssembly ?d . 
  }     
} 
'
powerAssemblyQueryEndpoint <- "http://localhost:3030/WBv13.4"
powerAssemblyQueryResults <- SPARQL(powerAssemblyQueryEndpoint,powerAssemblyQuery)
powerAssemblyQueryResults <- powerAssemblyQueryResults$results

if (nrow(powerAssemblyQueryResults) != 0)
    {
    if (sum(is.na(powerAssemblyQueryResults$sourceAssemblyID)) != 0)
        {
        print("The model does not comply with the rule.")
        print(sprintf("The %d power load assemblies presented above are not currently connected to any power sourcce assembly. Please correct the model.", sum(is.na(powerAssemblyQueryResults$sourceAssemblyID))))
        print("Suggestion: Look for omissions of those assemblies in the Function List Table.")
        row_to_keep = c(is.na(powerAssemblyQueryResults$sourceAssemblyID))
        powerAssemblyOrphan = powerAssemblyQueryResults[row_to_keep,]
        powerAssemblyOrphan %>%
          kable("html") %>%
          as.character() %>%
          display_html()
        }
    if (sum(is.na(powerAssemblyQueryResults$sourceAssemblyID)) == 0)
        {
        print("The model complies with the rule. No action needs to be performed.")
        }
    }
if (nrow(powerAssemblyQueryResults) == 0)
    {
    print("The pattern matching query returned no results. Please verify the value of the endpoint variable: powerAssemblyQueryEndpoint.")
    }

# Rule #3 : Each power load end circuit interface shall be connected to one power source end circuit interface

In [ ]:
powerInterfaceQuery <-
'
# Each power load end circuit interface shall be connected to one power source end circuit interface
# List all the power load end circuit interfaces

PREFIX oml: <http://def.seegrid.csiro.au/ontology/om/om-lite#>
PREFIX owl:   <http://www.w3.org/2002/07/owl#>
PREFIX rdfs:  <http://www.w3.org/2000/01/rdf-schema#>
PREFIX fse:   <http://imce.jpl.nasa.gov/discipline/fse/fse#>
PREFIX base:  <http://imce.jpl.nasa.gov/foundation/base/base#>
PREFIX analysis: <http://imce.jpl.nasa.gov/foundation/analysis/analysis#>
PREFIX mission: <http://imce.jpl.nasa.gov/foundation/mission/mission#>

SELECT DISTINCT ?powerLoadAssembly ?powerLoadAssemblyID ?powerLoadInterface ?functionID 
FROM <urn:x-arq:UnionGraph>
WHERE
{
    ?c a fse:Assembly ;
       base:hasAnnotations "Power load" ;
       fse:hasReferenceDesignator ?powerLoadAssemblyID ;
       base:hasCanonicalName ?powerLoadAssembly .
      
   
  optional 
  {
    ?a a <http://io.openceasar.firesat/efse/libraries/ECTypes#ID_oj74HBJ8czeoHVJe6MXGCvU94TQaXQzNM6wDjYko67NKxLaD5VQCW7zRrs2p5CxcF75J> ; 
    base:hasCanonicalName ?powerLoadInterface .
    
    ?b a fse:Function ;
    fse:hasFunctionNumber ?functionID ;
    mission:joins ?a .
    
    ?c mission:presents ?a .
   }
}
'
powerInterfaceQueryEndpoint <- "http://localhost:3030/WBv13.4"
powerInterfaceQueryResults <- SPARQL(powerInterfaceQueryEndpoint,powerInterfaceQuery)
powerInterfaceQueryResults <- powerInterfaceQueryResults$results

if (nrow(powerInterfaceQueryResults) != 0)
    {
    if (sum(is.na(powerInterfaceQueryResults$powerLoadInterface)) != 0)
        {
        print("The model does not comply with the rule.")
        print(sprintf("The %d power load assemblies presented above are missing a proper interface connection with a power source assembly. Please correct the model.", sum(is.na(powerInterfaceQueryResults$powerLoadInterface))))
        print("Suggestion: Look for omissions of those interfaces from the Function List Table. Start by the Assembly level, then the Function End List level, and at last the End Circuit Group level.")
        row_to_keep = c(is.na(powerInterfaceQueryResults$powerLoadInterface))
        powerInterfaceOrphan = powerInterfaceQueryResults[row_to_keep,]
        powerInterfaceOrphan %>%
          kable("html") %>%
          as.character() %>%
          display_html()
        }
    if (sum(is.na(powerInterfaceQueryResults$powerLoadInterface)) == 0)
        {
        print("The model complies with the rule. No action needs to be performed.")
        }
    }
if (nrow(powerInterfaceQueryResults) == 0)
    {
    print("The pattern matching query returned no results. Please verify the value of the endpoint variable: powerInterfaceQueryEndpoint.")
    }